In [1]:
import os
import logging
from datetime import datetime

In [2]:


import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hemch\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [4]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [5]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

#CODE_FOLDER = Path("code")
#sys.path.extend([f"./{CODE_FOLDER}"])

#DATA_FILEPATH = "../data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [6]:
import os

bucket = os.environ["BUCKET"]+'/sagemaker-course'
role = os.environ["ROLE"]

COMET_API_KEY = os.environ.get("COMET_API_KEY", None)
COMET_PROJECT_NAME = os.environ.get("COMET_PROJECT_NAME", None)

In [7]:
bucket

'beercafe-ml-bucket/sagemaker-course'

In [8]:
prefix = "iris"

In [9]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE batch_transform_op/
                           PRE data/
                           PRE model/


In [10]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-04-16 21:40:09        950 sagemaker-course/iris/data/iris_test.csv
2024-04-16 21:40:00       1900 sagemaker-course/iris/data/iris_train.csv


# Create Model from Artifacts Trained in Notebook 3

In [12]:
sagemaker_client = boto3.client("sagemaker")
sagemaker_runtime = boto3.client("runtime.sagemaker")

In [10]:
# This information is available in any training job of the model
model_img = "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest"
model_path = ("s3://beercafe-ml-bucket/sagemaker-course/iris/model/iris-xgboost-2024-04-16-22-38-06/output/model.tar.gz")

In [11]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost-2024-04-16-23-25-17


In [13]:
response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": model_img, "ModelDataUrl": model_path},
    ExecutionRoleArn=role,
)

print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:767398043272:model/iris-xgboost-2024-04-16-23-25-17', 'ResponseMetadata': {'RequestId': 'e0f151d3-e136-42ff-b402-f2800329812c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e0f151d3-e136-42ff-b402-f2800329812c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Tue, 16 Apr 2024 17:56:08 GMT'}, 'RetryAttempts': 0}}


# Setup Endpoint

In [14]:
ep_config_name = "tmp-ep-config-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_config_name)

tmp-ep-config-2024-04-16-23-26-54-058940


In [15]:
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": "version-1",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            # sever_less = ''
        },
    ],
)

print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:767398043272:endpoint-config/tmp-ep-config-2024-04-16-23-26-54-058940', 'ResponseMetadata': {'RequestId': '2866c7c1-5b98-4974-9e56-8a278f03dc85', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2866c7c1-5b98-4974-9e56-8a278f03dc85', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Tue, 16 Apr 2024 17:57:25 GMT'}, 'RetryAttempts': 0}}


In [16]:
ep_name = "tmp-ep-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-ep-2024-04-16-23-27-48-387085


In [17]:
response = sagemaker_client.create_endpoint(
    EndpointName=ep_name, EndpointConfigName=ep_config_name,
)
print(response)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:767398043272:endpoint/tmp-ep-2024-04-16-23-27-48-387085', 'ResponseMetadata': {'RequestId': 'db2e5b0f-539e-45b5-9b43-3859ddfe1c8d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'db2e5b0f-539e-45b5-9b43-3859ddfe1c8d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Tue, 16 Apr 2024 17:58:06 GMT'}, 'RetryAttempts': 0}}


In [18]:
waiter = sagemaker_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

Endpoint created


In [19]:
payload = "7.7, 3.0, 6.1, 2.3"

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=ep_name, ContentType="text/csv", Body=payload,
)

prediction = response["Body"].read().decode()
print(prediction)

2.0


# Very Imp- DELETE the endpoint so save cost

In [20]:
sagemaker_client.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3c63437c-2af3-4260-96fc-48c2df5563d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3c63437c-2af3-4260-96fc-48c2df5563d8',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 16 Apr 2024 18:02:50 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

# Delete Endpoint Config
This is not really necessary as there is no cost for keeping Endpoint Config but just for demo I am deleting from here.

In [21]:
sagemaker_client.delete_endpoint_config(EndpointConfigName=ep_config_name)

{'ResponseMetadata': {'RequestId': '35b3c55d-3f3b-41c8-9b86-7ed3c6b53049',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '35b3c55d-3f3b-41c8-9b86-7ed3c6b53049',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 16 Apr 2024 18:02:55 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}